In this code the sbert model is trained on cnn dailymail 8k dataset and store in the hf repo. Then import the model from the hf repo and run it. Need to upload the app.py and template folder in the collab content folder.

Install & imports

In [ ]:
!pip install -q sentence-transformers datasets rouge-score nltk

import os
import random
import numpy as np
from tqdm import tqdm

import torch
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

import nltk
nltk.download('punkt')

from rouge_score import rouge_scorer

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


  Preparing metadata (setup.py) ... done


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using device: cuda


Load CNN/DailyMail subset

In [ ]:
# Load train split from CNN/DailyMail
ds = load_dataset("cnn_dailymail", "3.0.0", split="train")

# How many articles to use for training (you can increase later)
NUM_ARTICLES = 8000   # try 2000 first if Colab is slow

ds = ds.shuffle(seed=42).select(range(NUM_ARTICLES))
print(ds)
print("Example keys:", ds.column_names)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 8000
})
Example keys: ['article', 'highlights', 'id']


Helper: score how “summary-like” a sentence is

In [ ]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def sentence_summary_score(sentence: str, summary: str) -> float:
    """
    Returns ROUGE-L F1 score between a sentence and the reference summary.
    Higher = more important/extractive sentence.
    """
    if not sentence.strip() or not summary.strip():
        return 0.0
    scores = scorer.score(summary, sentence)
    return scores["rougeL"].fmeasure


In [ ]:
import nltk

# Required for sentence tokenization
nltk.download('punkt')

# Sometimes needed, sometimes not—safe to include
try:
    nltk.download('punkt_tab')
except:
    print("punkt_tab not available, continuing anyway.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Build training examples for SBERT

In [ ]:
from nltk.tokenize import sent_tokenize
from sentence_transformers import InputExample

train_examples = []

max_sent_per_article = 30          # cap sentences per article for speed
min_positive_score = 0.2           # sentences above this are treated as positives
neg_keep_prob = 0.1                # keep only 10% of very low-score sentences

for i in tqdm(range(len(ds)), desc="Building training examples"):
    article = ds[i]["article"]
    summary = ds[i]["highlights"]

    sentences = sent_tokenize(article)
    if len(sentences) == 0:
        continue

    # Optional: limit sentences to avoid huge articles
    sentences = sentences[:max_sent_per_article]

    for sent in sentences:
        score = sentence_summary_score(sent, summary)

        # Keep strong positives
        if score >= min_positive_score:
            label = float(score)   # e.g. 0.2–1.0
            train_examples.append(InputExample(texts=[sent, summary], label=label))
        else:
            # For weak/non-summary sentences, keep only some as negatives
            if random.random() < neg_keep_prob:
                train_examples.append(InputExample(texts=[sent, summary], label=0.0))

print("Total training examples:", len(train_examples))
print("Example:", train_examples[0].texts, train_examples[0].label)


Building training examples: 100%|██████████| 8000/8000 [03:57<00:00, 33.73it/s]

Total training examples: 43308
Example: ['The bodies of married couple John and Audrey Cook were discovered alongside their daughter, Maureen, at the mobile home they shared on Tremarle Home Park in Camborne, west Cornwall.', 'John and .\nAudrey Cook were discovered alongside their daughter, Maureen .\nThey were found at Tremarle Home Park in Cornwall .\nInvestigators say the three died of carbon monoxide .\npoisoning .'] 0.5614035087719299


Load base SBERT + DataLoader

In [ ]:
from sentence_transformers import SentenceTransformer

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name, device=device)

batch_size = 16
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

train_loss = losses.CosineSimilarityLoss(model)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Train SBERT (fine-tuning)

In [ ]:
num_epochs = 1  # start with 1; you can try 2–3 later

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)

output_dir = "/content/sbert_extractive_cnn_dailymail"

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    show_progress_bar=True,
    output_path=output_dir  # saves at the end of training
)

print("Model saved to:", output_dir)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: prime-darkknight01 (prime-darkknight01-na) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.028200
1000,0.010900
1500,0.009900
2000,0.009100
2500,0.008900


Model saved to: /content/sbert_extractive_cnn_dailymail


Load fine-tuned model & use for extractive summarization

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from nltk.tokenize import sent_tokenize

fine_tuned_model = SentenceTransformer(output_dir, device=device)

def sbert_extractive_summary(article_text: str, top_n: int = 3) -> str:
    sentences = sent_tokenize(str(article_text))
    if len(sentences) == 0:
        return ""
    if len(sentences) <= top_n:
        return " ".join(sentences)

    # Encode each sentence with fine-tuned SBERT
    sentence_embeddings = fine_tuned_model.encode(sentences, convert_to_numpy=True)

    # Represent the whole article as mean of sentence embeddings
    article_embedding = np.mean(sentence_embeddings, axis=0, keepdims=True)

    # Compute similarity and pick top sentences
    sims = cosine_similarity(sentence_embeddings, article_embedding).ravel()
    top_idx = sims.argsort()[-top_n:][::-1]

    selected = [sentences[i] for i in sorted(top_idx)]
    return " ".join(selected)


Quick test on CNN/DailyMail sample

In [ ]:
idx = 0
art = ds[idx]["article"]
ref = ds[idx]["highlights"]

print("REFERENCE SUMMARY:\n", ref, "\n")
print("MODEL EXTRACTIVE SUMMARY:\n", sbert_extractive_summary(art, top_n=3))


REFERENCE SUMMARY:
 John and .
Audrey Cook were discovered alongside their daughter, Maureen .
They were found at Tremarle Home Park in Cornwall .
Investigators say the three died of carbon monoxide .
poisoning . 

MODEL EXTRACTIVE SUMMARY:
 upset.' 'Maureen . 'They .


In [ ]:
# 1️⃣ Install + imports (only once per runtime)
!pip install -q sentence-transformers nltk scikit-learn

import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# 2️⃣ Load your fine-tuned model
#   ➤ Use the same path you used in training:
#       output_dir = "/content/sbert_extractive_cnn_dailymail"
model_path = "/content/sbert_extractive_cnn_dailymail"   # change if needed
fine_tuned_model = SentenceTransformer(model_path, device=device)

# 3️⃣ Summarization function
def sbert_extractive_summary(article_text: str, top_n: int = 3) -> str:
    sentences = sent_tokenize(str(article_text))
    if len(sentences) == 0:
        return ""
    if len(sentences) <= top_n:
        return " ".join(sentences)

    # Encode each sentence with fine-tuned SBERT
    sentence_embeddings = fine_tuned_model.encode(sentences, convert_to_numpy=True)

    # Represent the whole article as mean of sentence embeddings
    article_embedding = np.mean(sentence_embeddings, axis=0, keepdims=True)

    # Compute similarity and pick top sentences
    sims = cosine_similarity(sentence_embeddings, article_embedding).ravel()
    top_idx = sims.argsort()[-top_n:][::-1]     # highest similarity first

    # Sort them back to original order
    selected = [sentences[i] for i in sorted(top_idx)]
    return " ".join(selected)

# 4️⃣ Give a news article as input
news = """
The city of Riverton has launched one of the largest public transportation overhauls in its history, aiming to cut traffic congestion and reduce carbon emissions over the next decade. After years of debate, the city council voted 9–2 in favor of a comprehensive plan that expands bus routes, introduces electric trams, and redesigns key intersections to prioritize pedestrians and cyclists. City officials say the project, estimated to cost 2.3 billion dollars, will be funded through a mix of municipal bonds, federal grants, and a small increase in local sales tax.

As part of the overhaul, more than forty new bus lines will be added, with special focus on low-income neighborhoods that previously had limited access to reliable transportation. Existing routes will be redesigned to reduce redundant paths and shorten overall travel time during peak hours. The new network will include several express services connecting residential suburbs to Riverton’s downtown business district, major hospitals, and the university campus. Transport planners estimate that average commute times could drop by up to twenty minutes for thousands of daily riders.

A major highlight of the plan is the introduction of an electric tram system that will run along two of the city’s busiest corridors. The trams are expected to carry up to two hundred passengers per trip and operate at intervals of six to eight minutes during rush hour. According to the transportation department, the electric fleet will be powered partly by a new solar farm being constructed on the outskirts of the city. Officials claim that once fully operational, the tram system alone could reduce annual greenhouse gas emissions by up to fifty thousand metric tons.

To make the city safer for non-motorized transport, Riverton will also build an additional one hundred kilometers of protected bike lanes and widen sidewalks in central commercial zones. Intersections with high accident rates will be redesigned with raised crosswalks, extended pedestrian signals, and clearer signage. Urban planners argue that these measures will not only protect cyclists and pedestrians but also create more vibrant street life that benefits local businesses. Several small business owners have already expressed cautious optimism, noting that fewer cars and more foot traffic could increase customer visits.

However, the plan is not without controversy. Some residents are worried about the short-term disruption caused by construction, which is expected to last at least five years. Road closures, detours, and noise from heavy machinery could make daily life more difficult for people living and working near major project sites. A group of local shopkeepers has raised concerns that prolonged construction could reduce sales, especially for businesses that rely on quick, drive-by customers. The city has promised compensation schemes and temporary signage to help affected businesses, but critics say the details are still vague.

Another point of debate is the increase in local sales tax, which will rise by half a percent to help finance the overhaul. Opponents argue that the tax hike will hit low- and middle-income families the hardest, especially as they already struggle with rising housing and utility costs. The mayor has responded by emphasizing that improved public transportation can save residents money in the long run by reducing fuel, parking, and car maintenance expenses. She also noted that federal funding is contingent on the city demonstrating its own financial commitment to the project.

Environmental groups have largely welcomed the overhaul, calling it a significant step toward making Riverton a greener and more livable city. They point out that transportation is currently the largest source of local greenhouse gas emissions, and that investing in public transit is more effective than relying solely on electric cars. Student organizations at Riverton University have also voiced support, saying the new network will make it easier and cheaper for them to commute between campus, internships, and part-time jobs. Some students have even proposed a discounted transit pass funded jointly by the city and the university.

City officials say the project will be rolled out in phases, with the first new bus routes expected to launch early next year. Construction of the tram lines and major street redesigns will begin later, once environmental impact assessments and final engineering studies are completed. The transportation department plans to collect detailed data on ridership, congestion, and air quality throughout the rollout to measure the plan’s success. If the targets are met, Riverton could become a model for other mid-sized cities looking to modernize their infrastructure in response to climate change and population growth.

"""

summary = sbert_extractive_summary(news, top_n=3)
print("SUMMARY:\n", summary)


Using device: cuda
SUMMARY:
 After years of debate, the city council voted 9–2 in favor of a comprehensive plan that expands bus routes, introduces electric trams, and redesigns key intersections to prioritize pedestrians and cyclists. Urban planners argue that these measures will not only protect cyclists and pedestrians but also create more vibrant street life that benefits local businesses. The mayor has responded by emphasizing that improved public transportation can save residents money in the long run by reducing fuel, parking, and car maintenance expenses.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!pip install -q huggingface_hub sentence-transformers

from huggingface_hub import notebook_login

notebook_login()   # ➜ paste your HF token from: https://huggingface.co/settings/tokens


In [ ]:
from sentence_transformers import SentenceTransformer

model_path = "/content/sbert_extractive_cnn_dailymail"   # same as output_dir
model = SentenceTransformer(model_path)

# ✅ change this to your username + model name
repo_id = "DarkKnight001/sbert-extractive-cnn-dailymail"

model.push_to_hub(
    repo_id,
    commit_message="Upload SBERT extractive summarization model trained on CNN/DailyMail",
)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...xoh8oey/model.safetensors:   1%|          |  558kB / 90.9MB            

'https://huggingface.co/DarkKnight001/sbert-extractive-cnn-dailymail/commit/35434f017840bc0a1248a1ddfa0d3e125654726e'

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("DarkKnight001/sbert-extractive-cnn-dailymail")


from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def sbert_extractive_summary(article_text: str, top_n: int = 3) -> str:
    sentences = sent_tokenize(str(article_text))
    if len(sentences) == 0:
        return ""
    if len(sentences) <= top_n:
        return " ".join(sentences)

    sentence_embeddings = model.encode(sentences, convert_to_numpy=True)
    article_embedding = np.mean(sentence_embeddings, axis=0, keepdims=True)

    sims = cosine_similarity(sentence_embeddings, article_embedding).ravel()
    top_idx = sims.argsort()[-top_n:][::-1]
    selected = [sentences[i] for i in sorted(top_idx)]
    return " ".join(selected)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

In [ ]:
news = """
The city of Riverton has launched one of the largest public transportation overhauls in its history, aiming to cut traffic congestion and reduce carbon emissions over the next decade. After years of debate, the city council voted 9–2 in favor of a comprehensive plan that expands bus routes, introduces electric trams, and redesigns key intersections to prioritize pedestrians and cyclists. City officials say the project, estimated to cost 2.3 billion dollars, will be funded through a mix of municipal bonds, federal grants, and a small increase in local sales tax.
"""

summary = sbert_extractive_summary(news, top_n=1)
print("SUMMARY:\n", summary)


NameError: name 'sbert_extractive_summary' is not defined

In [ ]:
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize
sent_tokenize("USA is bankrupt. India is growing fast.")


['USA is bankrupt.', 'India is growing fast.']

In [ ]:
!pip install flask tensorflow tensorflow_hub tensorflow_text joblib huggingface_hub pyngrok sentence-transformers nltk scikit-learn

import nltk
nltk.download("punkt")
nltk.download("punkt_tab")

from huggingface_hub import notebook_login
notebook_login()  # 🔐 login once – gives access to BOTH HF models




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("35dbBpjUOl9yP3NcpAPOe6BV3jV_6uYH5enpVmCC8m9a9sDhm")

from app import app   # imports app.py, downloads detection model + SBERT model

public_url = ngrok.connect(5000).public_url
print("Public URL:", public_url)

app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

📥 Downloading detection model from Hugging Face...


bert.pkl:   0%|          | 0.00/118k [00:00<?, ?B/s]

✔ Detection model file downloaded: /root/.cache/huggingface/hub/models--DarkKnight001--detection-bert-model/snapshots/5701416b55ff351f2eec041b685db4769a100ad9/bert.pkl
🔁 Loading sklearn classifier...
✔ Classifier loaded successfully!
📥 Loading SBERT summarization model from Hugging Face...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

✔ SBERT model loaded!


OSError: https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/3 does not appear to be a valid module.